# Setup

In [1]:
! pip install google-cloud-aiplatform --upgrade --quiet
! pip install google-cloud-discoveryengine --upgrade --quiet
! pip install google-cloud-documentai google-cloud-documentai-toolbox --upgrade --quiet
! pip install google-cloud-storage --upgrade --quiet

! pip install langchain-google-community --upgrade --quiet
! pip install langchain-google-vertexai --upgrade --quiet
! pip install langchain-google-community[vertexaisearch] --upgrade --quiet
! pip install langchain-google-community[docai] --upgrade --quiet

! pip install rich --upgrade --quiet
! pip install markdown --upgrade --quiet

  DEPRECATION: Building 'intervaltree' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'intervaltree'. Discussion can be found at https://github.com/pypa/pip/issues/6334
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.122.0 requires google-cloud-storage<3.0.0,>=1.32.0; python_version < "3.13", but you have google-cloud-storage 3.4.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-doc

In [2]:
import vertexai
from google.cloud import documentai
from google.cloud import discoveryengine

PROJECT_ID = "qwiklabs-gcp-04-b187b658bea8"  
REGION = "us-central1"  

vertexai.init(project=PROJECT_ID, location=REGION)
print(f"Vertex AI SDK initialized.")
print(f"Vertex AI SDK version = {vertexai.__version__}")
print(f"Document AI API version = {documentai.__version__}")
print(f"Discovery Engine API version = {discoveryengine.__version__}")

/opt/conda/lib/python3.10/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.5 is exactly one major version older than the runtime version 6.31.1 at google/protobuf/empty.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


Vertex AI SDK initialized.
Vertex AI SDK version = 1.122.0
Document AI API version = 2.35.0
Discovery Engine API version = 0.14.0


# Task 2. Initialize variables, import a Python utility file, and create resources
In this task, you initialize variables for cloud resources, download a Python utility file, and run code to create a Cloud Storage bucket, Vector Search index, and Document AI processor.



In [4]:
# Run the following in a cell to intialize variables and set names for your cloud storage bucket, index, index endpoint, and docai processor.

# Cloud storage buckets
GCS_BUCKET_URI = "gs://qwiklabs-gcp-04-b187b658bea8-bucket"  
GCS_OUTPUT_PATH = f"{GCS_BUCKET_URI}"  # DocAI Layout Parser Output Path
GCS_BUCKET_NAME = GCS_BUCKET_URI.replace("gs://", "")

# Vertex AI Vector Search
# parameter description here
# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.MatchingEngineIndex#google_cloud_aiplatform_MatchingEngineIndex_create_tree_ah_index
VS_INDEX_NAME = "qwiklabs-gcp-04-b187b658bea8-index"  
VS_INDEX_ENDPOINT_NAME = "qwiklabs-gcp-04-b187b658bea8-endpoint"  
VS_CONTENTS_DELTA_URI = f"{GCS_BUCKET_URI}/index/embeddings"
VS_DIMENSIONS = 768       # HERE
VS_APPROX_NEIGHBORS = 150 # HERE
VS_INDEX_UPDATE_METHOD = "STREAM_UPDATE"
VS_INDEX_SHARD_SIZE = "SHARD_SIZE_SMALL"
VS_LEAF_NODE_EMB_COUNT = 500
VS_LEAF_SEARCH_PERCENT = 80
VS_DISTANCE_MEASURE_TYPE = "DOT_PRODUCT_DISTANCE"    # HERE
VS_MACHINE_TYPE = "e2-standard-16"                   # HERE
VS_MIN_REPLICAS = 1
VS_MAX_REPLICAS = 1
VS_DESCRIPTION = "Index for DIY RAG with Vertex AI APIs"  

# Models
EMBEDDINGS_MODEL_NAME = "text-embedding-004"    # HERE
LLM_MODEL_NAME = "gemini-2.0-flash-001"         # HERE

# DocumentAI Processor
DOCAI_LOCATION = "us"  
DOCAI_PROCESSOR_NAME = "qwiklabs-gcp-04-b187b658bea8-docai-processor"  

# Enable/disable flags
# flag to create Google Cloud resources configured above
# refer to the notes after this cell
CREATE_RESOURCES = True  
# flag to run data ingestion
RUN_INGESTION = True  

In [5]:
# Run the following gcloud command in a new cell to download a Python file with some helpful utility functions.
!gcloud storage cp gs://qwiklabs-gcp-04-b187b658bea8/utils.py .

Copying gs://qwiklabs-gcp-04-b187b658bea8/utils.py to file://./utils.py
  Completed files 1/1 | 29.1kiB/29.1kiB                                        


The file utils.py should appear inside the file explorer. A good time to examine the file content would be while step 4 below is running, as it will take some time to complete. The file contains the following:
- DocAIParser, a class that uses the Document AI Layout Parser to parse PDF documents into chunks.
- CustomGCSDirectoryLoader, a class that loads documents from a Cloud Storage Bucket
- Utility methods for adding an index to Vector Search
- Utility methods for displaying rich content results

In [6]:
# Import the required library by adding the following to a new cell and running it with SHIFT+ENTER:
import utils

In [7]:
# Run the following code in a new cell to create necessary resources, then examine the code, and read the explanation that follows the code sample:

import hashlib
import uuid

from google.cloud import storage
from google.cloud import aiplatform
from google.cloud import documentai
from google.api_core.client_options import ClientOptions
from google.cloud.aiplatform import MatchingEngineIndex, MatchingEngineIndexEndpoint
from typing import List, Optional

def create_uuid(name: str) -> str:
    hex_string = hashlib.md5(name.encode("UTF-8")).hexdigest()
    return str(uuid.UUID(hex=hex_string))

def create_bucket(bucket_name: str) -> storage.Bucket:
    # create Cloud Storage bucket if does not exist
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    if bucket.exists():
        print(f"Bucket {bucket.name} exists")
        return bucket

    if not CREATE_RESOURCES:
        return bucket

    bucket = storage_client.create_bucket(bucket_name, project=PROJECT_ID)
    print(f"Bucket {bucket.name} created")
    return bucket

def create_index() -> Optional[MatchingEngineIndex]:
    index_names = [
        index.resource_name
        for index in MatchingEngineIndex.list(filter=f"display_name={VS_INDEX_NAME}")
    ]

    if len(index_names) > 0:
        vs_index = MatchingEngineIndex(index_name=index_names[0])
        print(
            f"Vector Search index {vs_index.display_name} exists with resource name {vs_index.resource_name}"
        )
        return vs_index

    if not CREATE_RESOURCES:
        print(
            f"CREATE_RESOURCES flag set to {CREATE_RESOURCES}. Skip creating resources"
        )
        return None

    print(f"Creating Vector Search index {VS_INDEX_NAME} ...")
    vs_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
        display_name=VS_INDEX_NAME,
        dimensions=VS_DIMENSIONS,
        approximate_neighbors_count=VS_APPROX_NEIGHBORS,
        distance_measure_type=VS_DISTANCE_MEASURE_TYPE,
        leaf_node_embedding_count=VS_LEAF_NODE_EMB_COUNT,
        leaf_nodes_to_search_percent=VS_LEAF_SEARCH_PERCENT,
        description=VS_DESCRIPTION,
        shard_size=VS_INDEX_SHARD_SIZE,
        index_update_method=VS_INDEX_UPDATE_METHOD,
        project=PROJECT_ID,
        location=REGION,
    )
    print(
        f"Vector Search index {vs_index.display_name} created with resource name {vs_index.resource_name}"
    )
    return vs_index

def create_index_endpoint() -> Optional[MatchingEngineIndexEndpoint]:
    endpoint_names = [
        endpoint.resource_name
        for endpoint in MatchingEngineIndexEndpoint.list(
            filter=f"display_name={VS_INDEX_ENDPOINT_NAME}"
        )
    ]

    if len(endpoint_names) > 0:
        vs_endpoint = MatchingEngineIndexEndpoint(index_endpoint_name=endpoint_names[0])
        print(
            f"Vector Search index endpoint {vs_endpoint.display_name} exists with resource name {vs_endpoint.resource_name}"
        )
        return vs_endpoint

    if not CREATE_RESOURCES:
        print(
            f"CREATE_RESOURCES flag set to {CREATE_RESOURCES}. Skip creating resources"
        )
        return None

    print(f"Creating Vector Search index endpoint {VS_INDEX_ENDPOINT_NAME} ...")
    vs_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
        display_name=VS_INDEX_ENDPOINT_NAME,
        public_endpoint_enabled=True,
        description=VS_DESCRIPTION,
        project=PROJECT_ID,
        location=REGION,
    )
    print(
        f"Vector Search index endpoint {vs_endpoint.display_name} created with resource name {vs_endpoint.resource_name}"
    )
    return vs_endpoint



# QUESTION
# THE INDEX OBJECT AND INDEXENDPOINT OBJECT ARE CREATED SEPARATELY
# THEN DEPLOYED TOGETHER USING 'endpoint.deploy_index',
# WHY are these not in 1 single call?
def deploy_index(
    index: MatchingEngineIndex, endpoint: MatchingEngineIndexEndpoint
) -> Optional[MatchingEngineIndexEndpoint]:
    index_endpoints = []
    if index is not None:
        index_endpoints = [
            (deployed_index.index_endpoint, deployed_index.deployed_index_id)
            for deployed_index in index.deployed_indexes
        ]

    if len(index_endpoints) > 0:
        vs_deployed_index = MatchingEngineIndexEndpoint(
            index_endpoint_name=index_endpoints[0][0]
        )
        print(
            f"Vector Search index {index.display_name} is already deployed at endpoint {vs_deployed_index.display_name}"
        )
        return vs_deployed_index

    if not CREATE_RESOURCES:
        print(
            f"CREATE_RESOURCES flag set to {CREATE_RESOURCES}. Skip creating resources"
        )
        return None

    print(
        f"Deploying Vector Search index {index.display_name} at endpoint {endpoint.display_name} ..."
    )
    deployed_index_id = (
        f'{VS_INDEX_NAME}_{create_uuid(VS_INDEX_NAME).split("-")[-1]}'.replace("-", "_")
    )
    vs_deployed_index = endpoint.deploy_index(
        index=index,
        deployed_index_id=deployed_index_id,
        display_name=VS_INDEX_NAME,
        machine_type=VS_MACHINE_TYPE,
        min_replica_count=VS_MIN_REPLICAS,
        max_replica_count=VS_MAX_REPLICAS,
    )
    print(
        f"Vector Search index {index.display_name} is deployed at endpoint {vs_deployed_index.display_name}"
    )
    return vs_deployed_index

def create_docai_processor(
    processor_display_name: str = DOCAI_PROCESSOR_NAME,
    processor_type: str = "LAYOUT_PARSER_PROCESSOR",
) -> Optional[documentai.Processor]:
    # Set the api_endpoint if you use a location other than 'us'
    opts = ClientOptions(api_endpoint=f"{DOCAI_LOCATION}-documentai.googleapis.com")
    docai_client = documentai.DocumentProcessorServiceClient(client_options=opts)
    parent = docai_client.common_location_path(PROJECT_ID, DOCAI_LOCATION)
    # Check if processor exists
    processor_list = docai_client.list_processors(parent=parent)
    processors = [
        processor.name
        for processor in processor_list
        if (
            processor.display_name == processor_display_name
            and processor.type_ == processor_type
        )
    ]

    if len(processors) > 0:
        docai_processor = docai_client.get_processor(name=processors[0])
        print(
            f"Document AI processor {docai_processor.display_name} is already created"
        )
        return docai_processor

    if not CREATE_RESOURCES:
        print(
            f"CREATE_RESOURCES flag set to {CREATE_RESOURCES}. Skip creating resources"
        )
        return None

    # Create a processor
    print(
        f"Creating Document AI processor {processor_display_name} of type {processor_type} ..."
    )
    docai_processor = docai_client.create_processor(
        parent=parent,
        processor=documentai.Processor(
            display_name=processor_display_name, type_=processor_type
        ),
    )
    print(
        f"Document AI processor {processor_display_name} of type {processor_type} is created."
    )
    return docai_processor


# @title Utility methods for adding index to Vertex AI Vector Search
def get_batches(items: List, n: int = 1000) -> List[List]:
    n = max(1, n)
    return [items[i : i + n] for i in range(0, len(items), n)]


def add_data(vector_store, chunks) -> None:
    if RUN_INGESTION:
        batch_size = 1000
        texts = get_batches([chunk.page_content for chunk in chunks], n=batch_size)
        metadatas = get_batches([chunk.metadata for chunk in chunks], n=batch_size)

        for i, (b_texts, b_metadatas) in enumerate(zip(texts, metadatas)):
            print(f"Adding {len(b_texts)} data points to index")
            is_complete_overwrite = bool(i == 0)
            vector_store.add_texts(
                texts=b_texts,
                metadatas=b_metadatas,
                is_complete_overwrite=is_complete_overwrite,
            )
    else:
        print("Skipping ingestion. Enable `RUN_INGESTION` flag")

if CREATE_RESOURCES:
    print("Creating new resources.")
else:
    print("Resource creation is skipped.")

# Create bucket if not exists
bucket = create_bucket(GCS_BUCKET_NAME)

# Create vector search index if not exists else return index resource name
vs_index = create_index()

# Create vector search index endpoint if not exists else return index endpoint resource name
vs_endpoint = create_index_endpoint()

# Deploy index to the index endpoint
deploy_index(vs_index, vs_endpoint)

# Create Document Layout Processor
docai_processor = create_docai_processor(processor_display_name=DOCAI_PROCESSOR_NAME)
PROCESSOR_NAME = docai_processor.name  # DocAI Layout Parser Processor Name

Creating new resources.
Bucket qwiklabs-gcp-04-b187b658bea8-bucket created
Creating Vector Search index qwiklabs-gcp-04-b187b658bea8-index ...
Vector Search index qwiklabs-gcp-04-b187b658bea8-index created with resource name projects/319136220147/locations/us-central1/indexes/6258888577254424576
Creating Vector Search index endpoint qwiklabs-gcp-04-b187b658bea8-endpoint ...
Vector Search index endpoint qwiklabs-gcp-04-b187b658bea8-endpoint created with resource name projects/319136220147/locations/us-central1/indexEndpoints/3074887621168594944
Deploying Vector Search index qwiklabs-gcp-04-b187b658bea8-index at endpoint qwiklabs-gcp-04-b187b658bea8-endpoint ...
Vector Search index qwiklabs-gcp-04-b187b658bea8-index is deployed at endpoint qwiklabs-gcp-04-b187b658bea8-endpoint
Creating Document AI processor qwiklabs-gcp-04-b187b658bea8-docai-processor of type LAYOUT_PARSER_PROCESSOR ...
Document AI processor qwiklabs-gcp-04-b187b658bea8-docai-processor of type LAYOUT_PARSER_PROCESSOR is 

This code creates a number of resources required by your pipeline. It uses Google Cloud libraries and the utility functions you imported earlier to create:
- A Cloud Storage Bucket to store raw text
- A Vector Search Index and endpoint to store and provide access to vectors
- A Document AI processor to parse and chunk documents

This would be a good time to examine the contents of utils.py, and to read the architecture and pipeline discussion at the beginning of the lab if you have not already done so.

Important: The code that you executed in step 4 creates a number of resources, including a Vector Search index with endpoint, and can take more than 25 minutes to complete. Wait for the cell to complete and the resources to be created before continuing.

Note: Resource creation is skipped if the CREATE_RESOURCES flag is set to False in the Initialize Variables section.

# Task 3. Ingest data
In this task, you read sample documents from a Cloud Storage bucket, parse them using the Document AI layout processor, extract chunks from the parsed document, generate embeddings using the Vertex AI Embeddings API, and add them to the Vertex AI Vector Search index.



**utils.py functions are used from task 3 onwards**

In [8]:
# Run the following code in a new cell to read the sample documents from a public Cloud Storage bucket.
# This code uses a class defined inside utils.py to load Alphabet investor reports for the years 2021, 2022, and 2023 from the public bucket.

loader = utils.CustomGCSDirectoryLoader(
    project_name=PROJECT_ID,
    bucket="cloud-samples-data",
    prefix="gen-app-builder/search/alphabet-investor-pdfs",
)

doc_blobs = loader.load(file_pattern=".*/202[1-3]")[:2]

PackageNotFoundError: No package metadata was found for langchain

In [ ]:
loader

In [ ]:
doc_blobs

In [ ]:
# Run the following code in a new cell to create a custom parser.
# Note: The DocAIParser class is defined inside utils.py. It uses the Document AI Layout Parser to convert blobs into layout-aware chunks.
# Layout Parser extracts document content elements like text, tables, and lists, and creates context-aware chunks that are useful for building RAG applications.

parser = utils.DocAIParser(
    project_id=PROJECT_ID,
    location=DOCAI_LOCATION,
    processor_name=PROCESSOR_NAME,
    gcs_output_path=GCS_OUTPUT_PATH,
)

In [ ]:
parser

In [ ]:
# Run the following code in a new cell to process the documents.
# Note: This cell should take less than 2 minutes to run to completion. Wait for it to finish before continuing.

docs = list(
    parser.batch_parse(
        doc_blobs,  # filter only last 40 for docs after 2020
        chunk_size=500,
        include_ancestor_headings=True,
    )
)

# At this point, you are ready to start converting chunks into embeddings. First, you examine two of the chunks.



In [ ]:
docs

In [ ]:
# Run the following code in a new cell to print the first and third chunks to the screen.

print("THE FIRST CHUNK:")
print(docs[1].page_content)
print()
print("THE THIRD CHUNK:")
print(docs[3].page_content)

# Note: Notice that the documents are parsed into different sections like title, subtitle, and even a Markdown table (an especially complex table with merged cells!).
# This makes it easy for retrieval as well for the downstream generation tasks. For example, a large language model (LLM) can now reason more effectively and more accurately.
# You have successfully chunked the document, but the chunks are still just text. Next, you create embeddings of the text chunks.


In [ ]:
# Run the following code in a new cell to define the model for creating embeddings:

from langchain_google_vertexai.embeddings import VertexAIEmbeddings

embedding_model = VertexAIEmbeddings(model_name=EMBEDDINGS_MODEL_NAME)


In [ ]:
# Run the following code to initialize the Vertex AI Vector Search retriever:

from langchain_google_vertexai.vectorstores.vectorstores import VectorSearchVectorStore

vector_store = VectorSearchVectorStore.from_components(
    project_id=PROJECT_ID,
    region=REGION,
    gcs_bucket_name=GCS_BUCKET_NAME,
    index_id=vs_index.resource_name,
    endpoint_id=vs_endpoint.resource_name,
    embedding=embedding_model,
    stream_update=True,
)

In [ ]:
# Finally, run the following code to use the add_data method in utils to store chunks as embeddings in the Vector Search index, and raw texts in the Cloud Storage bucket:

add_data(vector_store, docs)

# You have now retrieved source documents, processed as well as chunked them, embedded them into vectors, and upserted them into Vector Search.

# In the next task, you run searches against your vector store and generate grounded text.

# Task 4. Retrieve and rank results

In this task, you use Vertex AI Vector Search to retrieve the top-k relevant results, and then rerank them using the Vertex AI Ranking API based on chunk content and semantic similarity to the query.

![RAG](https://cdn.qwiklabs.com/7SEpiASYyaQBABk34SnpQPGVHqBK0c6rzMjilbO9eag%3D)

The Ranking API takes a list of documents and reranks them based on their relevance to a given query. Unlike embeddings, which focus on semantic similarity between a document and a query, the Ranking API provides precise relevance scores that indicate how well each document answers the query.

The Ranking API is stateless, meaning you don’t need to index documents beforehand. You simply pass in the query and the list of documents. This makes it ideal for reranking results retrieved from Vector Search or other search solutions to improve the overall quality of search results.

Note: For more information, refer to the Improve search and RAG quality with ranking API reference documentation.
https://cloud.google.com/generative-ai-app-builder/docs/ranking


In [ ]:
# Run the following code in a new cell to define and combine retrievers using Vector Search and the Vertex AI Ranking API:

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_google_community import VertexAIRank

# Instantiate the VertexAIReranker with the SDK manager
reranker = VertexAIRank(
    project_id=PROJECT_ID,
    location_id="global",
    ranking_config="default_ranking_config",
    title_field="source",  # metadata field to preserve with reranked results
    top_n=5,
)

basic_retriever = vector_store.as_retriever(
    search_kwargs={"k": 5}
)  # fetch top 5 documents

# Create the ContextualCompressionRetriever with the VertexAIRanker as a Reranker
retriever_with_reranker = ContextualCompressionRetriever(
    base_compressor=reranker, base_retriever=basic_retriever
)

# Note: By prioritizing semantically relevant documents, the Ranking API improves the LLM's context, leading to more accurate and well-reasoned answers.
# In the next step, you compare the Retriever Results and the Reranked Results side-by-side to see the improvement.



In [ ]:
# Run the following code in a new cell to use the get_sxs_comparison method in utils to view original and reranked results side-by-side inside your notebook:

reranked_results = utils.get_sxs_comparison(
    simple_retriever=basic_retriever,
    reranking_api_retriever=retriever_with_reranker,
    query="what was google cloud revenue in 2023 ?",
    search_kwargs={"k": 5},
)

# Note: You have retrieved the most relevant facts from your indexed source data. Now you can configure a RAG chain that follows this pipeline: query -> vector search -> retrieve documents -> LLM -> rerank documents -> check grounding.

## Check Grounding API
The Check Grounding API returns an overall support score between 0 and 1, indicating how well an answer candidate aligns with a given set of facts. It also provides citations for the facts supporting each claim in the text.

A **claim is considered perfectly grounded** only if it is **fully supported by one or more facts. Partial support does not qualify as grounded**. For example, the claim Google was founded by Larry Page and Sergey Brin in 1975 is ungrounded—the founders are correct, but the date is wrong.
**In this version of the API, each sentence is treated as a single claim.**

You can use the Check Grounding API to evaluate any piece of text—human or AI-generated. A common use case is verifying LLM responses against a known fact set.

The API is optimized for low latency (<500ms), making it suitable for real-time chatbot integration.
It also returns citations and a support score, enabling applications to highlight grounded content and filter out hallucinations using a citation threshold.

Note: For more information, refer to the Check grounding with RAG documentation. https://cloud.google.com/generative-ai-app-builder/docs/check-grounding


In [ ]:
# Run the following code in a new cell to configure a retreiver from the vector store you defined earlier.

from typing import List

from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from langchain.docstore.document import Document
from langchain_core.runnables import chain

from langchain_google_vertexai import VertexAI
from langchain.prompts import PromptTemplate

from langchain_google_community import VertexAICheckGroundingWrapper

from rich import print

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [ ]:
# Run the following code in a new cell to configure the LLM with a prompt template to generate the answer.

llm = VertexAI(model_name=LLM_MODEL_NAME, max_output_tokens=1024)
template = """
Answer the question based only on the following context:
{context}

Question:
{query}
"""
prompt = PromptTemplate.from_template(template)

create_answer = prompt | llm

In [ ]:
# Run the following code in a new cell to define a wrapper to call Vertex AI Check Grounding API on the generated answer:

output_parser = VertexAICheckGroundingWrapper(
    project_id=PROJECT_ID,
    location_id="global",
    grounding_config="default_grounding_config",
    top_n=3,
)




In [ ]:
# Finally, complete the workflow by defining a QA chain with a check on the grounding of the result:

@chain
def check_grounding_output_parser(answer_candidate: str, documents: List[Document]):
    return output_parser.with_config(configurable={"documents": documents}).invoke(
        answer_candidate
    )





# this 'workflow' is used in the fn below
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "query": RunnablePassthrough()}
)

@chain
def qa_with_check_grounding(query):
    docs = setup_and_retrieval.invoke(query)
    answer_candidate = create_answer.invoke(docs)
    check_grounding_output = check_grounding_output_parser.invoke(
        answer_candidate, documents=docs["context"]
    )
    return check_grounding_output

# Your entire pipeline is now in place. You have ingested data, parsed it into chunks, converted chunks to embeddings and stored them in Vector Search. You have then created retrievers to rerank results and check their grounding. Finally, you created a chain that included a prompt template and called the Ranking and Check Grounding APIs, respectively, to get the most relevant result and rate the level of grounding.

# With everything in place, it is time to run a query and invoke the chain.



In [ ]:
# Run the following code in a new cell to pass the query "what was google cloud revenue in Q1 2021?" to your qa_with_check_grounding chain:

result = qa_with_check_grounding.invoke("what was google cloud revenue in Q1 2021?")
print(result)

# Note: You should get a response from the check grounding service, including a support_score measuring grounding accuracy, as well as cited chunks, sources, and the answer to the query.


In [ ]:
# Run the following code in a new cell to view the answer to the query, the source link, and a citation:

utils.display_grounded_generation(result)


Your query result should resemble the following.

![Query Result](https://cdn.qwiklabs.com/Oq%2B9MJ2Q8jGhHLKzWbtLKd9SznHKfMG%2BokEcWMsfKOg%3D)

In [ ]:
# Run the following code to run a second query against the chain, and display grounded results:

result = qa_with_check_grounding.invoke(
    "what are the main influencing factors on Alphabet revenue in Q1 2021 ?"
)
utils.display_grounded_generation(result)